In [ ]:
import codebase.match_data as match
import codebase.settings as settings
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib as plt
import seaborn as sns
import codebase.analysis_functions as af
import string
import sklearn
import numpy as np
m  = match.MatchData('1263466')
%load_ext autoreload
%autoreload 2

In [ ]:
fifth_ashes_test = af.pre_transform_comms(m)


In [ ]:
fours = fifth_ashes_test[fifth_ashes_test['isFour'] == True]
sixes = fifth_ashes_test[fifth_ashes_test['isSix'] == True]
wickets = fifth_ashes_test[fifth_ashes_test['isWicket'] == True]

In [ ]:
fours = fours[['commentTextItems', 'isFour']]
sixes = sixes[['commentTextItems', 'isSix']]
wickets = wickets[['commentTextItems', 'isWicket']]

In [ ]:
fours.head()

In [ ]:
fours.shape

In [ ]:
def verify_if_string_exists(string, value):
    if string in value:
        return True
    return False

def verify_fours(value):
    return verify_if_string_exists('four', value)

def verify_six(value):
    return verify_if_string_exists('six', value)

def verify_wicket(value):
    return verify_if_string_exists('wicket', value)

In [ ]:
fours['verify'] = fours['commentTextItems'].map(verify_fours)

In [ ]:
fours.head()

In [ ]:
fours[fours['verify'] == True]

In [ ]:
sixes['verify'] = sixes['commentTextItems'].map(verify_six)
wickets['verify'] = wickets['commentTextItems'].map(verify_wicket)

In [ ]:
sixes[sixes['verify'] == True]

In [ ]:
wickets[wickets['verify'] == True]

In [ ]:
def create_dictionary(words, reverse = False):
    dictionary = {}
    for word in words:
        if word in dictionary:
            dictionary[word] += 1
        else:
            dictionary[word] = 1
    return dict(sorted(dictionary.items(), key=lambda item: item[1], reverse= not reverse))

In [ ]:
fours_words = []
for entry in fours['commentTextItems']:
    entry = entry.translate(str.maketrans('', '', string.punctuation))
    entry = entry.split()
    fours_words += [e.lower() for e in entry]

In [ ]:
fours_dict = create_dictionary(fours_words)
fours_dict

In [ ]:
fours_dict = af.create_vocabulary(fours, m)

In [ ]:
wickets_dict = af.create_vocabulary(wickets, m)
wickets_dict

In [ ]:
player_list = af.get_player_map(m, name_tag='known_as').values()
list(player_list)
player_list = [name.lower() for player in player_list for name in player.split()]
player_list

In [ ]:
sixes_vocab = af.create_vocabulary(sixes, m)

In [ ]:
sixes_vocab

In [ ]:
sixes_freq = pd.Series(sixes_vocab)
sixes_freq

In [ ]:
sixes_freq = af.series_to_df(sixes_freq, ['word', 'freq'])

In [ ]:
af.graph_seaborn_barplot(sixes_freq, x = sixes_freq['word'], y = sixes_freq['freq'])

In [ ]:
def graph_vocab(vocab_dict):
    vocab_dict = pd.Series(vocab_dict)
    vocab_dict = af.series_to_df(vocab_dict, ['Words', 'Frequency'])
    af.graph_seaborn_barplot(vocab_dict, vocab_dict['Words'], vocab_dict['Frequency'])

In [ ]:
# graph_vocab(fours_dict)

In [ ]:
# graph_vocab(wickets_dict)

In [ ]:
test_commentary = fifth_ashes_test.loc[:, ['commentTextItems', 'isFour', 'isSix', 'isWicket']]

In [ ]:
test_commentary.head()

In [ ]:
test_commentary.loc[:, 'noEvent'] = False

In [ ]:
for index, row in test_commentary.iterrows():
    print(int(row.loc[['isFour', 'isSix', 'isWicket']].max()))

In [ ]:
for index, row in test_commentary.iterrows():
    if row['isWicket'] == False:
        if row['isFour'] == False:
            if row['isSix'] == False:
                test_commentary.at[index, 'noEvent'] = True

In [ ]:
test_commentary.tail()

In [ ]:
test_commentary_categorial = test_commentary[['isWicket', 'isFour', 'isSix', 'noEvent']].idxmax(axis=1)
test_commentary_categorial.tail()

In [ ]:
test_commentary_categorial = test_commentary_categorial.to_frame(name='Ball_Event')
test_commentary_categorial['commentTextItems'] = test_commentary['commentTextItems']

In [ ]:
test_commentary_categorial.tail()

In [ ]:
def categorical_mapping(value):
    mapping = {
        'isFour': 'four',
        'isSix': 'six',
        'isWicket': 'wicket',
        'noEvent': 'no-event'
    }
    try:
        return mapping[value]
    except KeyError:
        return value

In [ ]:
test_commentary_categorial['Ball_Event'] = test_commentary_categorial['Ball_Event'].map(categorical_mapping)
test_commentary_categorial.tail()

In [ ]:
train_data = list(test_commentary_categorial['commentTextItems'])
train_data[:5]


In [ ]:
train_labels = list(test_commentary_categorial['Ball_Event'])
train_labels

In [ ]:
train_label_names = set(train_labels)
train_label_names

In [ ]:
training_set = sklearn.utils.Bunch(data = train_data, labels = train_labels, names = train_label_names)



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
x_train = count_vect.fit_transform(training_set.data)
x_train.shape

In [ ]:
count_vect.vocabulary_.get('bat')

In [ ]:
count_vect.vocabulary_

In [ ]:
def cat_to_num(labels, label_names):
    mapping = {cat:i for i,cat in enumerate(label_names)}
    labels_nums = [mapping[label] for label in labels]
    return labels_nums

In [ ]:
training_set.labels = cat_to_num(training_set.labels, training_set.names)
training_set.labels[-5:]


In [ ]:
training_set.labels[-5:]

In [ ]:
training_set.labels = np.array(training_set.labels)
training_set.labels[-5:]

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tf_transformer  = TfidfTransformer()
x_train_tfidf = tf_transformer.fit_transform(x_train)
x_train_tfidf.shape

In [ ]:
test_commentary = fifth_ashes_test.loc[:, ['commentTextItems', 'isFour', 'isSix', 'isWicket']]

In [ ]:
test_commentary_labels = af.create_labels(test_commentary, ['isFour', 'isSix', 'isWicket'], 'noEvent')
test_commentary_labels.tail()

In [ ]:
train_set = af.package_data(test_commentary_labels['commentTextItems'], test_commentary_labels['labels'])

In [ ]:
train_set.data[-5:]

In [ ]:
train_set.labels[-5:]

In [ ]:
train_set.label_names

In [ ]:
fifth_ashes_test.columns

In [ ]:
fifth_ashes_test['batsmanRuns'].head()

In [ ]:
singles = fifth_ashes_test[fifth_ashes_test['batsmanRuns'] == 1]
doubles = fifth_ashes_test[fifth_ashes_test['batsmanRuns'] == 2]
threes = fifth_ashes_test[fifth_ashes_test['batsmanRuns'] == 3]
fives = fifth_ashes_test[fifth_ashes_test['batsmanRuns'] == 5]

In [ ]:
singles['commentTextItems']

In [ ]:
doubles['commentTextItems'].size

In [ ]:
threes['commentTextItems'].iloc[3]

In [ ]:
dots = fifth_ashes_test[fifth_ashes_test['batsmanRuns'] == 0]

In [ ]:
fifth_ashes_test.groupby('batsmanRuns').count()/fifth_ashes_test.shape[0]

In [ ]:
fifth_ashes_test.shape

In [ ]:
dots = fifth_ashes_test['batsmanRuns'] == 0

In [ ]:
dots.head()

In [ ]:
af.createDummies(fifth_ashes_test)

In [ ]:
fifth_ashes_test.columns

In [ ]:
fifth_ashes_test_categorical = af.create_labels(fifth_ashes_test, ['isFour', 'isSix', 'isWicket', 'isOne', 'isTwo', 'isThree'], 'noEvent')